In [1]:
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import config


In [2]:
driver = webdriver.Chrome()  # Mac用の記述
driver.implicitly_wait(10)


In [3]:
# SBIネオモバイルのログインページへアクセス
url_login = "https://trade.sbineomobile.co.jp/login"
driver.get(url_login)
time.sleep(3)  # ページに遷移する前に次の処理が実行されないようにするため


In [4]:
# ログインフォームの要素を取得する
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
login_btn = driver.find_element(By.ID, "neo-login-btn")

# 念のためテキストボックスの中身を空にする
username.clear()
password.clear()

# テキストボックスに値を入力する
username.send_keys(config.SBI_USERNAME)
password.send_keys(config.SBI_PASSWORD)

# ログインボタンをクリックする
login_btn.click()
time.sleep(1)


In [5]:
# SBIネオモバイルのポートフォリオ
url_portfolio = "https://trade.sbineomobile.co.jp/account/portfolio"
driver.get(url_portfolio)
time.sleep(3)


In [6]:
# 銘柄一覧の要素を取得する
div_element = driver.find_element(By.CLASS_NAME, "sp-main")

# 無限スクロールが終わるまでスクロールする
last_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
while True:
    driver.execute_script(
        "arguments[0].scrollTo(0, arguments[0].scrollHeight);", div_element
    )
    time.sleep(2)
    new_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
    if new_height == last_height:
        break
    last_height = new_height


In [53]:
# ページのhtmlを取得してパースする
html = driver.page_source.encode("utf-8")
parsed_html = BeautifulSoup(html, "html.parser")


In [54]:
# 保有銘柄の証券コード、銘柄名をそれぞれSeriesにする

# 証券コード
stock_code_list = []
tickers = parsed_html.find_all(class_="ticker")

for ticker in tickers:
    stock_code = ticker.get_text().strip()  # 抽出したテキストに空白がある場合は除去する
    stock_code_list.append(stock_code)

ser_stock_code = pd.Series(stock_code_list)

# 銘柄名
stock_name_list = []
names = parsed_html.find_all(class_="name")

for name in names:
    stock_name = name.get_text().strip()  # 抽出したテキストに空白がある場合は除去する
    stock_name_list.append(stock_name)

ser_stock_name = pd.Series(stock_name_list)


In [60]:
# 全銘柄の現在値〜預り区分をデータフレームのリストにする
table = parsed_html.find_all("table")
list_df_tables = pd.read_html(str(table))

# リストのデータフレームを一つに結合する
df_all_stock = pd.DataFrame()
for df_table in list_df_tables:
    row = df_table.T[1:2]
    df_all_stock = pd.concat([df_all_stock, row], axis=0)

# インデックスを振り直す
df_all_stock = df_all_stock.reset_index(drop=True)


In [61]:
# 証券コード、銘柄名をデータフレームに結合する
df_all_stock = pd.concat([ser_stock_code, ser_stock_name, df_all_stoÏck], axis=1)

# カラム名を付け直す
df_all_stock.columns = [
    "コード",
    "名称",
    "現在値/前日比",
    "保有数量",
    "（うち売却注文中）",
    "評価損益率",
    "平均取得単価",
    "預り区分",
]


,コード,名称,現在値/前日比,保有数量,（うち売却注文中）,評価損益率,平均取得単価,預り区分
0,1434,ＪＥＳＣＯホールディングス,510円 / 8 + 1.59%,38株,0株,22.01%,418 円,特定
1,1518,三井松島ホールディングス,"3,170円 / 55 + 1.77%",7株,0株,-1.4%,"3,215 円",特定
2,1775,富士古河Ｅ＆Ｃ,"3,850円 / 30 + 0.79%",3株,0株,46.72%,"2,624 円",特定
3,1820,西松建設,"3,500円 / -5 -0.14%",29株,0株,-3.74%,"3,636 円",特定
4,1835,東鉄工業,"2,685円 / 38 + 1.44%",71株,0株,20.19%,"2,234 円",特定
...,...,...,...,...,...,...,...,...
89,9603,エイチ・アイ・エス,"1,987円 / -25 -1.24%",7株,0株,-1.68%,"2,021 円",特定
90,9698,クレオ,891円 / 2 + 0.22%,88株,0株,3.97%,857 円,特定
91,9831,ヤマダホールディングス,469円 / -4 -0.85%,100株,0株,21.19%,387 円,特定
92,9880,イノテック,"1,365円 / 3 + 0.22%",53株,0株,2.79%,"1,328 円",特定
